In [1]:
import shutil
import os

def save_retinanet_to_savedmodel(
    backbone_name,
    weights_path,
    num_classes,
    export_path,
    preprocessing_mode,
    nms=True,
    class_specific_filter=True,
    anchor_params=None
):
    import keras
    import numpy as np
    import tensorflow as tf
    from keras_retinanet import models as rn_models
    from keras_retinanet.models import retinanet as rn_retinanet
    from keras_retinanet.utils import anchors as rn_anchors
    from keras_retinanet import layers as rn_layers
    
    class RetinaNetPreprocessingLayer(keras.layers.Layer):
        def __init__(self, preprocessing_mode, *args, **kwargs):
            super().__init__(*args, **kwargs)
            assert preprocessing_mode in ['caffe', 'tf'], 'Please provide a valid preprocessing mode (caffe or tf).'
            self.preprocessing_mode = preprocessing_mode

        def call(self, x, **kwargs):
            if self.preprocessing_mode == 'caffe':
                return x - keras.backend.variable([103.939, 116.779, 123.68])
            elif self.preprocessing_mode == 'tf':
                return (x - 127.5) / (127.5)
            else:
                raise NotImplementedError

    # 1. Build and load the model
    model = rn_models.backbone(backbone_name=backbone_name).retinanet(num_classes=num_classes)
    model.load_weights(weights_path)

    # 2. Convert to inference model
    model = rn_models.convert_model(
        model=model,
        nms=nms,
        class_specific_filter=class_specific_filter,
        anchor_params=anchor_params
    )
    
    # 3. Add helpful names to the output layers.
    outputs = [
        keras.layers.Lambda(lambda x: keras.backend.tf.identity(x, name='output_boxes'))(model.outputs[0]),
        keras.layers.Lambda(lambda x: keras.backend.tf.identity(x, name='output_scores'))(model.outputs[1]),
        keras.layers.Lambda(lambda x: keras.backend.tf.identity(x, name='output_labels'))(model.outputs[2]),
    ]
    model = keras.models.Model(inputs=model.inputs, outputs=outputs, name='retinanet-bbox')

    # 4. Add scaling as part of the network. This makes life just a little easier later on.
    input_layer = keras.layers.Input(batch_shape=model.input_shape, name='images')
    preprocessing_model = keras.models.Model(
        inputs=input_layer,
        outputs=RetinaNetPreprocessingLayer(preprocessing_mode=preprocessing_mode)(input_layer)
    )
    model = keras.models.Model(inputs=input_layer, outputs=model(preprocessing_model.outputs))

    # 5. Save the model to disk.
    if os.path.isdir(export_path):
        shutil.rmtree(export_path)
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    signature = tf.saved_model.predict_signature_def(
        inputs={'images': model.input},
        outputs=dict(zip([o.name[:-2] for o in model.outputs], model.outputs))
    )

    sess = keras.backend.get_session()
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tf.saved_model.tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    return builder.save()

def convert_retinanet_savedmodel_to_tfjs(tf_savedmodel_dir, tfjs_dir):
    from tensorflowjs.converters import converter
    # 8. In a brand new kernel, convert the saved TensorFlow model to TensorFlowJS format.
    if os.path.isdir(tfjs_dir):
        shutil.rmtree(tfjs_dir)

    converter.tf_saved_model_conversion_v2.convert_tf_saved_model(
        saved_model_dir=tf_savedmodel_dir,
        output_dir=tfjs_dir,
        signature_def='predict',
        saved_model_tags='serve',
        strip_debug_ops=True
    )
    model_path = os.path.join(tfjs_dir, 'model.json')
    with open(model_path, 'r') as rf:
        model_text = rf.read().replace('DT_INT64', 'DT_INT32')
    with open(model_path, 'w') as wf:
        wf.write(model_text)

In [ ]:
"""
# Javascript
const modelPath = tfn.io.fileSystem('test_assets/purify_tfjs/model.json');
const classes = [
    'BELLY_EXPOSED',
    'BUTTOCKS_EXPOSED',
    'FEMALE_BREAST_EXPOSED',
    'FEMALE_BREAST_COVERED',
    'FEMALE_GENITALIA_EXPOSED',
    'FEMALE_GENITALIA_COVERED',
    'MALE_GENITALIA_EXPOSED',
    'MALE_BREAST_EXPOSED'
];

# Python
save_retinanet_to_savedmodel(
    backbone_name='resnet101',
    weights_path='Purify/model/model.h5',
    num_classes=8,
    export_path='purify_savedmodel',
    preprocessing_mode='caffe'
)
convert_retinanet_savedmodel_to_tfjs(
    tf_savedmodel_dir='purify_savedmodel',
    tfjs_dir='purify_tfjs'
)
"""

In [ ]:
# import cv2
# import numpy as np

# X = np.array([cv2.cvtColor(cv2.imread('test3.jpg'), cv2.COLOR_BGR2RGB)])
# y = model.predict(X)